<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent Communication and Planning

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code.

In this notebook, we demonstrate how to use multiple agents to work together and accomplish a task which requires finding info from the web and coding. `AssistantAgent` is an LLM-based agent that can write Python code (in a Python coding block) for a user to execute for a given task. `UserProxyAgent` is an agent which serves as a proxy for a user to execute the code written by `AssistantAgent`. We further create an planning agent for the assistant agent to consult. The planning agent is a variation of the `AssistantAgent`.

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [autogen] option:
```bash
pip install flaml[autogen]
```

In [1]:
# %pip install flaml[autogen]==2.0.0rc3

## Set your API Endpoint

* The [`config_list_openai_aoai`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_openai_aoai) function tries to create a list of configurations using Azure OpenAI endpoints and OpenAI endpoints. 

It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.


In [2]:
from flaml import oai

config_list = oai.config_list_openai_aoai()

## Construct Agents

We construct the planning agent named "planner" and a user proxy agent for the planner named "planner_user". We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will never ask for human feedback. We define `ask_planner` function to send a message to planner and return the suggestion from the planner.

In [3]:
from flaml.autogen.agent import AssistantAgent, UserProxyAgent

planner = AssistantAgent(
    name="planner",
    config_list=config_list,
    model="gpt-4",
    # the default system message of the AssistantAgent is overwritten here
    system_message="You are a helpful AI assistant. You suggest coding and reasoning steps for another AI assistant to accomplish a task. Do not suggest concrete code. For any action beyond writing code or reasoning, convert it to a step which can be implemented by writing code. For example, the action of browsing the web can be implemented by writing code which reads and prints the content of a web page. Finally, inspect the execution result. If the plan is not good, suggest a better plan. If the execution is wrong, point out the error and suggest a fix."
)
planner_user = UserProxyAgent(
    name="planner_user",
    max_consecutive_auto_reply=0,  # terminate without auto-reply
    human_input_mode="NEVER",
)

def ask_planner(message):
    planner.receive(message, planner_user)
    return planner._oai_conversations[planner_user.name][-1]["content"]


We construct the assistant agent and the user proxy agent. We specify `human_input_mode` as "TERMINATE" in the user proxy agent, which will ask for feedback when it receives a "TERMINATE" signal from the assistant agent. We set the `functions` in `AssistantAgent` and `function_map` in UserProxyAgent to use the created `ask_planner` function.

In [4]:
# create an AssistantAgent instance named "assistant"
assistant = AssistantAgent(
    name="assistant",
    request_timeout=600,
    seed=42,
    # Excluding azure openai endpoints from the config list.
    # Change to `exclude="openai"` to exclude openai endpoints, or remove the `exclude` argument to include both.
    config_list=oai.config_list_openai_aoai(exclude="aoai"),
    model="gpt-4-0613",  # make sure the endpoint you use supports the model
    functions=[
        {
            "name": "ask_planner",
            "description": "ask planner to: 1. get a plan, 2. when you finish the execution, verify the execution result of the plan and potentially suggest new plan.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "question to ask planner. Make sure the question include enough context, such as the code and the execution result.",
                    },
                },
                "required": ["message"],
            },
        }
    ],
)

# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    name="user",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
    work_dir="planning",
    function_map={"ask_planner": ask_planner},
)

## Perform a task

We invoke the `receive()` method of the coding agent to start the conversation. When you run the cell below, you will be prompted to provide feedback after the coding agent sends a "TERMINATE" signal in the end of the message. If you don't provide any feedback (by pressing Enter directly), the conversation will finish. Before the "TERMINATE" signal, the user proxy agent will try to execute the code suggested by the coding agent on behalf of the user.

In [5]:
# the assistant receives a message from the user, which contains the task description
assistant.receive(
    """Find a good first issue in flaml's github repo and suggest a fix.""",
    user
)

user (to assistant):

Find a good first issue in flaml's github repo and suggest a fix.

--------------------------------------------------------------------------------
assistant (to user):

***** Suggested function Call: ask_planner *****
Arguments: 
{
  "message": "Find a good first issue in flaml's Github repo"
}
************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
planner_user (to planner):

Find a good first issue in flaml's Github repo

--------------------------------------------------------------------------------
planner (to planner_user):

1. Write code to send an HTTP request to access flaml's GitHub repository API to obtain issue list data. The API URL: `https://api.github.com/repos/microsoft/flaml/issues`

2. Add query parameters to filter the issues. Specifically, use the labels query parameter to filter for "good first issue".

user (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
--------------------------------------------------------------------------------
Title:  Do not print data split method when user provides X_val in AutoML
URL:  https://github.com/microsoft/FLAML/issues/1099
Detail:  Currently, the fit function always prints a log message saying

> INFO - Data split method: {some method chosen automatically or provided by the user as a setting/argument}

![image](https://github.com/microsoft/FLAML/assets/47984759/e9f57e92-4dcc-487a-835e-66e15389d886)


In the cases where `X_val` is provided by the user,[`task.prepare_data`](https://github.com/microsoft/FLAML/blob/e3ca95bf8a5cc9c86638dab8ef0eb44bc1f3ee6f/flaml/automl/task/generic_task.py#L348) doesn't [call `self._train_test_split`](https://github.com/microsoft/FLAML/blob/e3ca95bf8a5cc9c86638dab8ef0eb44bc1f3ee6f/flaml/automl/task/generic_task.py#L514), i.e., no further splitting is done. 

In this instances, printing the data split me

When the assistant needs to consult the planner, it suggests a function call to `ask_planner`. When this happens, a line like the following will be displayed:

***** Suggested function Call: ask_planner *****
